## Libreries

In [1]:
import pandas as pd
import numpy as np

## Data

In [2]:
root_files = "../data/labels/"

In [3]:
sample_2017 = pd.read_excel("../data/sample_2017.xlsx", sheet_name=-1, index_col=0)
sample_2017

,pregunta_id,respuesta_id,pregunta,respuesta
10752,3653,67161,Un libro tiene 132 páginas y en cada una hay 2...,Esta mal porque 132+263=593
22502,4581,85815,¿Qué aprendiste de la actividad del viernes?,ha ver las cajas porque las cajas chicas eran ...
21926,4548,85184,"Sí Antonia tiene 15 caramelos en su casa, pasa...",Antonia tiene 10 caramelos llo lo resolbi \nAN...
16296,4087,76406,La señora Magdalena camina 30 minutos en la ma...,estamal porque es una ora \ncon20minutos
3698,3106,54633,Juan tiene 7 años y su hermano Pedro tiene 5 a...,12
...,...,...,...,...
2334,2963,52065,Joaquin reparte láminas a sus amigos. Le entre...,se queda con 12
8753,3487,63800,"María repartió, en partes iguales, 12 dulces e...",48
27460,5093,95242,Durante la sesión de conecta ideas María Jesús...,"es correcto, porque yo lo \nmultiplique por 4..."
21270,4495,84226,Amanda compró 2 chocolates a $135 cada uno y u...,no esta mal por que pago con mil pesos


In [4]:
names_of_labelers = ["Ignacia", "Diana", "Jessica", "Francisco", "Jorge", "Karla", "Natalie", ]
checks = "Check (Tipo de pregunta)  Check (¿Es incoherente?)  Check (¿Independiente?)".split("  ")

In [5]:
%%time
dic_labelers = {}
for name in names_of_labelers:
    df_label = pd.read_excel(root_files+name+".xlsx", sheet_name=-1)
    df_label = df_label[(df_label[checks[0]] == "Listo!") & (df_label[checks[1]] == "Listo!") & (df_label[checks[2]] == "Listo!")]
    df_label = df_label["respuesta_id  pregunta_id  Tipo de pregunta  ¿Es incoherente?  ¿Independiente?".split("  ")]
    df_label = df_label.fillna(0)
    dic_labelers[name] = df_label.set_index("respuesta_id")

Wall time: 8.08 s


In [144]:
table_0 = []
for ix, (k, v) in enumerate(dic_labelers.items()):
    x = v["¿Es incoherente?"].value_counts()
    num = v.shape[0]
    prop = np.round(100*x[1]/(x[0]+x[1]), 1)
#     print("labeler:",  ix, "\b, number of labels:", num, "\b, prop_C1:", prop)
    table_0.append({"labeler": ix, "num of labels": num, "prop C1": prop})
print(pd.DataFrame(table_0).set_index("labeler").T.to_latex(index=False))

\begin{tabular}{rrrrrrr}
\toprule
     0 &      1 &      2 &     3 &     4 &     5 &      6 \\
\midrule
1084.0 & 2995.0 & 2220.0 & 451.0 & 100.0 & 737.0 & 2859.0 \\
  23.4 &   28.0 &   25.6 &  25.1 &  68.0 &  24.3 &   33.8 \\
\bottomrule
\end{tabular}



<ipython-input-144-74163a7825a6>:8: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pd.DataFrame(table_0).set_index("labeler").T.to_latex(index=False))


In [6]:
agg_17 = dic_labelers[names_of_labelers[0]].reset_index()["respuesta_id  ¿Es incoherente?".split("  ")].rename(columns={'¿Es incoherente?': names_of_labelers[0]})
for name in names_of_labelers[1:]:
    df = dic_labelers[name].reset_index()
    df = df["respuesta_id  ¿Es incoherente?".split("  ")]
    df = df.rename(columns={'¿Es incoherente?': name})
    agg_17 = agg_17.merge(df, on="respuesta_id", how="outer")
agg_17 = agg_17.set_index("respuesta_id")

In [55]:
group_A = agg_17.drop(columns=["Jorge", "Karla", "Francisco"])
subgroup_A = group_A.dropna()
subgroup_A_p = subgroup_A[subgroup_A.nunique(axis=1) == 1]
A_test = sample_2017.set_index("respuesta_id").copy()
A_test["label"] = subgroup_A_p.mean(axis=1)
A_test = A_test.dropna()
A_test = A_test.reset_index()
A_test = A_test.rename(columns={
    "respuesta_id": "id",
    "pregunta_id": "Q_id",
    "pregunta": "Q", 
    "respuesta": "A"
})
A_test = A_test.set_index("id")
A_test

,Q_id,Q,A,label
id,,,,
85815,4581,¿Qué aprendiste de la actividad del viernes?,ha ver las cajas porque las cajas chicas eran ...,0.0
64812,3535,¿Cómo creen ustedes que se reproducen las célu...,El proceso de división de una célula en dos cé...,0.0
84148,4491,¿ Qué frase para mejorar le darías tú a un am...,debes practicar los ejerccios,0.0
88565,4734,¿Qué aprendiste en la actividad del día viernes?,Nose porque no fui. A donde fueron ustedes😛😛😛😛😜,1.0
51688,2944,Diego compró 2 sopaipillas a $120 cada una. Pa...,las sopaipillas,1.0
...,...,...,...,...
54685,3109,Lorena dice que al multiplicar 3x0 le da como...,estamal poreque es 0,0.0
87324,4665,Alicia comió 2/9 de un chocolate. Ella dice qu...,esta mal Alicia por que la cantidad es 7/8 de...,0.0
73798,3946,El cuarto básico visitará el zoológico. Si son...,nose\n,1.0


In [147]:
group_A.dropna().shape[0]

1078

In [97]:
A_test["label"].value_counts()

0.0    541
1.0    136
Name: label, dtype: int64

In [104]:
136/(136+541)

0.20088626292466766

In [148]:
s = group_A.dropna().sum(axis=1)
((s == 0) | (s == 4)).value_counts()

True     677
False    401
dtype: int64

In [99]:
677/(677+401)

0.6280148423005566

In [150]:
(s>=1).value_counts()

False    541
True     537
dtype: int64

In [151]:
537/(541+537)

0.49814471243042674

In [96]:
for i in range(1, 4):
    print(i, (s[s>=i] == 4).value_counts(), sep="\n")

1
False    401
True     136
dtype: int64
2
False    175
True     136
dtype: int64
3
True     136
False     78
dtype: int64


In [103]:
136/(136+401), 136/(175+136), 136/(78+136)

(0.2532588454376164, 0.43729903536977494, 0.6355140186915887)

In [8]:
agg_17_Q = dic_labelers[names_of_labelers[0]].reset_index()["pregunta_id  respuesta_id  Tipo de pregunta".split("  ")].rename(columns={'Tipo de pregunta': names_of_labelers[0]})
for name in names_of_labelers[1:]:
    df = dic_labelers[name].reset_index()["respuesta_id  Tipo de pregunta".split("  ")].rename(columns={'Tipo de pregunta': name})
    agg_17_Q = agg_17_Q.merge(df, on="respuesta_id", how="outer")
agg_17_Q = agg_17_Q.set_index("respuesta_id")

In [9]:
group_A_Q = agg_17_Q.drop(columns=["Jorge", "Karla", "Francisco"])
subgroup_A_Q = group_A_Q.dropna()
subgroup_A_Q_p = subgroup_A_Q[subgroup_A_Q.drop(columns="pregunta_id").nunique(axis=1) == 1]
Q_test = sample_2017.set_index("respuesta_id").copy()
Q_test["label"] = subgroup_A_Q_p.drop(columns="pregunta_id").mean(axis=1)
Q_test = Q_test.dropna()
Q_test = Q_test.reset_index()
Q_test = Q_test.loc[Q_test["pregunta_id"].drop_duplicates().index]
Q_test = Q_test.drop(columns="respuesta_id  respuesta".split("  "))
Q_test = Q_test.rename(columns={
    "pregunta_id": "id", 
    "pregunta": "Q"
})
Q_test = Q_test.set_index("id")
Q_test

,Q,label
id,,
4900,Alejandra tiene una caja de 4x3x2 cm llena de ...,2.0
4436,¿Qué te dijo el monito y qué te parece?,0.0
3535,¿Cómo creen ustedes que se reproducen las célu...,5.0
4555,"Amalia compró 1,75 cm de elástico y Antonieta ...",3.0
3742,Don Antonio compró 3 cajas de cereales a $ 673...,3.0
...,...,...
4445,¿Qué te dijo el monito y qué te parece?,0.0
4689,La profesora pregunta a sus alumnos: ¿la figu...,3.0
4663,Antonella compró 23 dulces para su curso. Rega...,3.0


## Save dataframes

In [93]:
Q_test.to_excel("../data/test_task_Q.xlsx")
A_test.to_excel("../data/test_task_C1.xlsx")